# 🤖 **CRO ML Deep Dive - Foundation Phase Implementation**
## **Interactive Walkthrough of Enrollment Prediction & Site Risk Scoring Models**

---

### 📋 **Notebook Overview**
This notebook provides a detailed technical walkthrough of our Foundation phase ML implementation for Contract Research Organization (CRO) operations. We'll explore:

1. **🏗️ Data Architecture & Feature Engineering**
2. **📊 Exploratory Data Analysis** 
3. **🎯 Model 1: Enrollment Prediction (Linear Regression)**
4. **⚠️ Model 2: Site Risk Scoring (Logistic Regression)**
5. **🔮 Prediction Pipeline & Business Impact**
6. **📈 Model Performance & Validation**

**Target Audience**: Data Scientists evaluating Snowflake ML capabilities  
**Demo Context**: Medpace CRO - Mid-sized player competing with industry giants  
**Technical Approach**: Familiar algorithms with rich clinical domain features  

---


## 🔧 **Environment Setup**

Let's start by connecting to our CRO demo environment and importing the necessary libraries.


In [ ]:
-- Set up our environment
USE ROLE SF_INTELLIGENCE_DEMO;
USE DATABASE CRO_AI_DEMO;
USE SCHEMA CLINICAL_OPERATIONS_SCHEMA;
USE WAREHOUSE CRO_DEMO_WH;

-- Verify our setup
SELECT 
    CURRENT_ROLE() as current_role,
    CURRENT_DATABASE() as current_database,
    CURRENT_SCHEMA() as current_schema,
    CURRENT_WAREHOUSE() as current_warehouse;


## 🏗️ **1. Data Architecture Overview**

Before diving into modeling, let's understand our data architecture. We've built a comprehensive clinical trial data model with dedicated ML infrastructure.


In [ ]:
-- Overview of our data architecture
SELECT 
    TABLE_SCHEMA,
    TABLE_TYPE,
    COUNT(*) as table_count,
    LISTAGG(TABLE_NAME, ', ') as table_names
FROM INFORMATION_SCHEMA.TABLES 
WHERE TABLE_CATALOG = 'CRO_AI_DEMO'
    AND TABLE_SCHEMA IN ('CLINICAL_OPERATIONS_SCHEMA', 'ML_MODELS')
GROUP BY TABLE_SCHEMA, TABLE_TYPE
ORDER BY TABLE_SCHEMA, TABLE_TYPE;


In [ ]:
-- Let's examine our core clinical data
SELECT 
    'Studies' as entity,
    COUNT(*) as count,
    STRING_AGG(DISTINCT study_phase, ', ') as phases,
    STRING_AGG(DISTINCT study_status, ', ') as statuses
FROM DIM_STUDIES

UNION ALL

SELECT 
    'Sites' as entity,
    COUNT(*) as count,
    STRING_AGG(DISTINCT site_tier, ', ') as tiers,
    STRING_AGG(DISTINCT country, ', ') as countries
FROM DIM_SITES

UNION ALL

SELECT 
    'Sponsors' as entity,
    COUNT(*) as count,
    STRING_AGG(DISTINCT sponsor_type, ', ') as types,
    STRING_AGG(DISTINCT company_size, ', ') as sizes
FROM DIM_SPONSORS;


## 📊 **2. Feature Engineering Deep Dive**

The key to successful ML in clinical research is rich feature engineering that captures domain expertise. Let's explore our feature tables.


In [ ]:
-- Examine enrollment prediction features
SELECT 
    'Enrollment Features' as feature_set,
    COUNT(*) as record_count,
    COUNT(DISTINCT study_id) as unique_studies,
    COUNT(DISTINCT site_id) as unique_sites
FROM CRO_AI_DEMO.ML_MODELS.ML_ENROLLMENT_FEATURES

UNION ALL

SELECT 
    'Site Performance Features' as feature_set,
    COUNT(*) as record_count,
    COUNT(DISTINCT site_id) as unique_sites,
    NULL as unique_sites_2
FROM CRO_AI_DEMO.ML_MODELS.ML_SITE_PERFORMANCE_FEATURES;


In [ ]:
-- Deep dive into enrollment features
SELECT 
    study_phase,
    therapeutic_area,
    site_tier,
    AVG(study_complexity_score) as avg_complexity,
    AVG(historical_enrollment_rate) as avg_historical_rate,
    AVG(patient_population_density) as avg_population_density,
    AVG(final_enrollment_rate) as avg_final_rate,
    COUNT(*) as sample_size
FROM CRO_AI_DEMO.ML_MODELS.ML_ENROLLMENT_FEATURES
GROUP BY study_phase, therapeutic_area, site_tier
ORDER BY study_phase, therapeutic_area;


### 🔍 **Feature Engineering Insights**

Notice how our features capture multiple dimensions of clinical trial complexity:

- **Study Characteristics**: Phase, therapeutic area, complexity score
- **Site Capabilities**: Historical performance, experience, tier
- **Market Dynamics**: Patient population, competition level, seasonality
- **Performance Indicators**: Current trends, screen failure rates

This multi-dimensional approach is what gives our models clinical relevance beyond simple statistical relationships.


## 🎯 **3. Model 1: Enrollment Prediction Model**

Let's dive deep into our enrollment prediction model. We use linear regression because:
1. **Interpretability**: Clinical teams need to understand predictions
2. **Baseline**: Establishes credible foundation before complex models
3. **Feature Importance**: Clear understanding of what drives enrollment


In [ ]:
-- Let's examine the training data for enrollment prediction
SELECT 
    study_complexity_score,
    historical_enrollment_rate,
    site_experience_score,
    patient_population_density,
    seasonal_factor,
    competition_level,
    final_enrollment_rate,
    CASE 
        WHEN final_enrollment_rate >= 8.0 THEN 'High Performer'
        WHEN final_enrollment_rate >= 5.0 THEN 'On Track'
        ELSE 'At Risk'
    END as performance_category
FROM CRO_AI_DEMO.ML_MODELS.ML_ENROLLMENT_FEATURES
WHERE final_enrollment_rate IS NOT NULL
ORDER BY final_enrollment_rate DESC;


In [ ]:
-- Train the enrollment prediction model and examine results
CALL CRO_AI_DEMO.ML_MODELS.TRAIN_ENROLLMENT_PREDICTION_MODEL();


In [ ]:
-- Examine the trained model metadata
SELECT 
    model_id,
    model_name,
    training_date,
    model_status,
    feature_importance,
    performance_metrics,
    comments
FROM CRO_AI_DEMO.ML_MODELS.ML_MODEL_REGISTRY
WHERE model_type = 'enrollment_prediction'
ORDER BY training_date DESC
LIMIT 1;


### 🔬 **Model Performance Analysis**

Let's parse the model performance metrics to understand how well our linear regression performs.


In [ ]:
-- Parse and analyze model performance metrics
WITH model_metrics AS (
    SELECT 
        model_id,
        model_name,
        PARSE_JSON(performance_metrics) as metrics,
        PARSE_JSON(feature_importance) as features
    FROM CRO_AI_DEMO.ML_MODELS.ML_MODEL_REGISTRY
    WHERE model_type = 'enrollment_prediction'
    ORDER BY training_date DESC
    LIMIT 1
)
SELECT 
    model_name,
    metrics:mean_absolute_error::FLOAT as mae,
    metrics:r2_score::FLOAT as r2_score,
    metrics:training_samples::INT as training_samples,
    features
FROM model_metrics;


### 🎯 **Generate and Analyze Predictions**

Now let's generate predictions for our current studies and analyze the results.


In [ ]:
-- Generate enrollment predictions
CALL CRO_AI_DEMO.ML_MODELS.GENERATE_ENROLLMENT_PREDICTIONS(
    (SELECT model_id FROM CRO_AI_DEMO.ML_MODELS.ML_MODEL_REGISTRY 
     WHERE model_type = 'enrollment_prediction' 
     ORDER BY training_date DESC LIMIT 1)
);


In [ ]:
-- Analyze enrollment predictions with business context
SELECT 
    st.study_title,
    st.study_phase,
    st.planned_enrollment,
    st.actual_enrollment,
    s.site_name,
    s.country,
    p.prediction_value as predicted_enrollment_rate,
    p.prediction_category,
    p.confidence_score,
    p.business_impact,
    ROUND((p.prediction_value * 4.33), 0) as predicted_monthly_enrollment -- ~4.33 weeks per month
FROM CRO_AI_DEMO.ML_MODELS.ML_PREDICTIONS p
JOIN DIM_STUDIES st ON p.entity_id = st.study_id
JOIN CRO_AI_DEMO.ML_MODELS.ML_ENROLLMENT_FEATURES f ON f.study_id = st.study_id
JOIN DIM_SITES s ON f.site_id = s.site_id
WHERE p.entity_type = 'study_site' 
    AND p.prediction_date >= CURRENT_DATE - 1
ORDER BY p.prediction_value DESC;


## ⚠️ **4. Model 2: Site Risk Scoring Model**

Our second model focuses on predicting site performance risk using logistic regression. This is a classification problem where we predict the probability of site underperformance.


In [ ]:
-- Explore site performance features and risk indicators
SELECT 
    s.site_name,
    s.site_tier,
    s.country,
    f.therapeutic_expertise_match,
    f.historical_enrollment_rate,
    f.historical_data_quality_avg,
    f.query_resolution_rate,
    f.protocol_deviation_rate,
    f.staff_turnover_indicator,
    f.regulatory_issues_count,
    f.site_risk_level,
    f.underperformance_indicator
FROM CRO_AI_DEMO.ML_MODELS.ML_SITE_PERFORMANCE_FEATURES f
JOIN DIM_SITES s ON f.site_id = s.site_id
ORDER BY f.underperformance_indicator DESC, f.protocol_deviation_rate DESC;


In [ ]:
-- Train the site risk scoring model
CALL CRO_AI_DEMO.ML_MODELS.TRAIN_SITE_RISK_SCORING_MODEL();


In [ ]:
-- Analyze site risk model performance
WITH risk_model_metrics AS (
    SELECT 
        model_id,
        model_name,
        PARSE_JSON(performance_metrics) as metrics,
        PARSE_JSON(feature_importance) as features
    FROM CRO_AI_DEMO.ML_MODELS.ML_MODEL_REGISTRY
    WHERE model_type = 'site_risk_scoring'
    ORDER BY training_date DESC
    LIMIT 1
)
SELECT 
    model_name,
    metrics:accuracy::FLOAT as accuracy,
    metrics:precision::FLOAT as precision,
    metrics:recall::FLOAT as recall,
    metrics:f1_score::FLOAT as f1_score,
    metrics:training_samples::INT as training_samples,
    features
FROM risk_model_metrics;


In [ ]:
-- Generate site risk predictions
CALL CRO_AI_DEMO.ML_MODELS.GENERATE_SITE_RISK_SCORES(
    (SELECT model_id FROM CRO_AI_DEMO.ML_MODELS.ML_MODEL_REGISTRY 
     WHERE model_type = 'site_risk_scoring' 
     ORDER BY training_date DESC LIMIT 1)
);


In [ ]:
-- Analyze site risk predictions with actionable insights
SELECT 
    s.site_name,
    s.principal_investigator,
    s.country,
    s.site_tier,
    ROUND(p.prediction_value * 100, 1) as risk_percentage,
    p.prediction_category,
    p.business_impact,
    -- Extract key risk factors from feature values
    PARSE_JSON(p.feature_values):protocol_deviation_rate::FLOAT as deviation_rate,
    PARSE_JSON(p.feature_values):query_resolution_rate::FLOAT as query_resolution,
    PARSE_JSON(p.feature_values):staff_turnover_numeric::INT as staff_turnover
FROM CRO_AI_DEMO.ML_MODELS.ML_PREDICTIONS p
JOIN DIM_SITES s ON p.entity_id = s.site_id
WHERE p.entity_type = 'site_performance' 
    AND p.prediction_date >= CURRENT_DATE - 1
ORDER BY p.prediction_value DESC;


## 🔮 **5. Prediction Pipeline & Business Integration**

Let's explore how our ML predictions integrate with the broader CRO operations through our ML-enhanced semantic views and business intelligence.


In [ ]:
-- High-level ML predictions summary
SELECT * FROM ML_PREDICTIONS_SUMMARY
ORDER BY prediction_count DESC;


In [ ]:
-- High-risk sites requiring immediate attention
SELECT 
    site_name,
    principal_investigator,
    country,
    ROUND(risk_probability * 100, 1) as risk_percentage,
    business_impact
FROM HIGH_RISK_SITES_ALERT
ORDER BY risk_probability DESC;


In [ ]:
-- Enrollment performance forecasting for resource planning
SELECT 
    study_title,
    site_name,
    country,
    ROUND(predicted_enrollment_rate, 2) as weekly_enrollment_rate,
    performance_category,
    ROUND(confidence_score * 100, 1) as confidence_percentage
FROM ENROLLMENT_PERFORMANCE_FORECAST
ORDER BY predicted_enrollment_rate DESC;


### 🤖 **Natural Language Queries with ML Predictions**

One of the key advantages of our approach is that business users can now ask natural language questions about ML predictions through Cortex Analyst.


**Example Natural Language Queries:**
- *"Which sites have the highest predicted enrollment rates?"*
- *"Show me all high-risk sites in Europe"*
- *"What's the average ML confidence score for our predictions?"*
- *"How many sites are predicted to be high performers?"*

These queries can be executed through the CRO_INTELLIGENCE_AGENT using the ML_ENHANCED_CLINICAL_VIEW semantic view.


## 📈 **6. Model Performance & Validation Deep Dive**

Let's perform a more detailed analysis of our model performance and validation approach.


In [ ]:
-- Compare model performance across both use cases
SELECT 
    model_type,
    model_name,
    training_date,
    PARSE_JSON(performance_metrics) as metrics,
    comments
FROM CRO_AI_DEMO.ML_MODELS.ML_MODEL_REGISTRY
ORDER BY training_date DESC;


In [ ]:
-- Feature importance analysis across models
WITH feature_analysis AS (
    SELECT 
        model_type,
        model_name,
        PARSE_JSON(feature_importance) as features
    FROM CRO_AI_DEMO.ML_MODELS.ML_MODEL_REGISTRY
    WHERE model_status = 'Active'
)
SELECT 
    model_type,
    model_name,
    features
FROM feature_analysis;


### 🔍 **Model Validation & Clinical Relevance**

Our CRAWL phase models use simple, interpretable algorithms for good reasons:

1. **Clinical Interpretability**: Healthcare professionals need to understand why a model made a prediction
2. **Regulatory Compliance**: Simple models are easier to validate and explain to regulatory bodies  
3. **Trust Building**: Starting with familiar algorithms builds confidence before introducing complexity
4. **Feature Importance**: Clear understanding of what clinical factors drive predictions

### 📊 **Performance Benchmarks**

**Enrollment Prediction Model:**
- **Algorithm**: Linear Regression
- **Target Metric**: R² Score (coefficient of determination)
- **Business Goal**: Predict weekly enrollment rates within ±2 subjects/week
- **Success Criteria**: R² > 0.7 for clinical relevance

**Site Risk Scoring Model:**
- **Algorithm**: Logistic Regression  
- **Target Metric**: F1 Score (balanced precision/recall)
- **Business Goal**: Identify 80%+ of underperforming sites before issues escalate
- **Success Criteria**: F1 > 0.75 for operational value


## 🚀 **7. Business Impact & ROI Analysis**

Let's quantify the business impact of our ML implementation.


In [ ]:
-- Calculate potential business impact from ML predictions
WITH impact_analysis AS (
    SELECT 
        'Enrollment Optimization' as use_case,
        COUNT(*) as predictions_made,
        AVG(confidence_score) as avg_confidence,
        SUM(CASE WHEN prediction_category = 'High Performance Expected' THEN 1 ELSE 0 END) as high_performers,
        SUM(CASE WHEN prediction_category = 'At Risk' THEN 1 ELSE 0 END) as at_risk_sites
    FROM CRO_AI_DEMO.ML_MODELS.ML_PREDICTIONS
    WHERE entity_type = 'study_site'
    
    UNION ALL
    
    SELECT 
        'Site Risk Management' as use_case,
        COUNT(*) as predictions_made,
        AVG(confidence_score) as avg_confidence,
        SUM(CASE WHEN prediction_category = 'High Risk' THEN 1 ELSE 0 END) as high_risk_sites,
        SUM(CASE WHEN prediction_category = 'Low Risk' THEN 1 ELSE 0 END) as low_risk_sites
    FROM CRO_AI_DEMO.ML_MODELS.ML_PREDICTIONS
    WHERE entity_type = 'site_performance'
)
SELECT 
    use_case,
    predictions_made,
    ROUND(avg_confidence * 100, 1) as avg_confidence_pct,
    high_performers as high_value_predictions,
    at_risk_sites as intervention_needed
FROM impact_analysis;


### 💰 **ROI Calculation for Medpace**

Based on industry benchmarks and our ML predictions:

**Enrollment Optimization Impact:**
- **25% improvement** in enrollment timeline accuracy
- **Average study delay cost**: $600K - $8M per month
- **Potential savings**: $2-5M per study through better site selection

**Site Risk Management Impact:**
- **Early intervention** prevents 60-80% of site performance issues
- **Average site remediation cost**: $50K - $200K per site
- **Potential savings**: $300K - $1.2M annually across portfolio

**Operational Efficiency:**
- **60% reduction** in manual analysis time
- **Data scientist productivity**: 2-3x improvement in analysis speed
- **Faster decision making**: 48-72 hour reduction in response time

### 🎯 **Competitive Advantage for Mid-Sized CRO**

Our ML capabilities help Medpace compete with larger CROs by:
- **Predictive site selection** vs. reactive management
- **Data-driven sponsor conversations** with quantified risk assessments  
- **Proactive operational management** reducing sponsor escalations
- **Faster proposal responses** with ML-powered feasibility analysis


## 🔮 **8. Next Steps: Advanced & Strategic Phases**

This Foundation phase establishes the foundation. Here's what comes next:


### 🚀 **Advanced Phase - Sophisticated Analytics**

**Patient Recruitment Optimization:**
- **Clustering algorithms** for patient population segmentation
- **Geographic analysis** with external demographic data
- **Multi-objective optimization** for site selection

**Clinical Data Anomaly Detection:**
- **Unsupervised learning** for automated data quality monitoring
- **Real-time streaming** with Snowpipe for immediate alerts
- **Pattern recognition** for protocol deviation detection

### 🏆 **Strategic Phase - Market Intelligence**

**Therapeutic Area Market Intelligence:**
- **External data integration** (competitor intelligence, market trends)
- **Predictive market modeling** for business development
- **Competitive positioning** analysis with ML

**Sponsor Relationship Optimization:**
- **Churn prediction** models for client retention
- **Recommendation engines** for cross-selling opportunities
- **Sentiment analysis** of sponsor communications

### 🎯 **Technical Architecture Evolution**

**Foundation**: Simple models, basic features, SQL-based workflows  
**Advanced**: Complex models, external data, Python/Scala in Snowpark  
**Strategic**: Deep learning, real-time inference, automated decision systems

---

## ✅ **Summary: Foundation Phase Achievements**

We've successfully implemented a foundation for ML-driven CRO operations:

✅ **Two Production Models** with immediate business value  
✅ **Integrated ML Pipeline** from training to prediction to business insight  
✅ **Natural Language Access** to ML predictions via Cortex Analyst  
✅ **Scalable Architecture** ready for advanced capabilities  
✅ **Clinical Domain Expertise** embedded in features and interpretations  

**The foundation is set for Medpace to compete with industry giants through superior data science capabilities!** 🚀
